In [1]:
import numpy as np
import pandas as pd
import spacy 
import nltk
import sklearn

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
book_list = pd.read_pickle('../data/scary_books.pkl')

In [4]:
book_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063 entries, 0 to 2062
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             2063 non-null   object 
 1   author            2063 non-null   object 
 2   description       2063 non-null   object 
 3   publisher         1766 non-null   object 
 4   customer_reviews  2042 non-null   float64
 5   stars             2042 non-null   float64
dtypes: float64(2), object(4)
memory usage: 96.8+ KB


In [5]:

book_list.isnull().sum()

title                 0
author                0
description           0
publisher           297
customer_reviews     21
stars                21
dtype: int64

In [6]:
book_list['stars']=book_list['stars'].fillna(book_list['stars'].mean())

In [7]:
book_list['customer_reviews']=book_list['customer_reviews'].fillna(0)

In [8]:
book_list['publisher']=book_list['publisher'].fillna('Self')

In [9]:
book_list.isnull().sum()

title               0
author              0
description         0
publisher           0
customer_reviews    0
stars               0
dtype: int64

In [10]:
book_list["id"] = book_list.index + 1

In [11]:
book_list.head()

,title,author,description,publisher,customer_reviews,stars,id
0,A Time to Kill: A Novel [Kindle Edition],John Grisham,"Don’t miss a sneak peek of Sycamore Row, John ...",Dell,918.0,4.5,1
1,Failure to Appear (J. P. Beaumont Novel) [Kind...,J. A. Jance,A shattering tale of tragedy and horror from t...,William Morrow; Reprint edition,34.0,4.8,2
2,Twelve Years a Slave [Kindle Edition],Solomon Northup,"First published in 1853, Twelve Years a Slave ...",HarperPerennial Classics,1374.0,4.6,3
3,Without Due Process (J. P. Beaumont Novel) [Ki...,J. A. Jance,Top ten New York Times bestselling author J.A....,William Morrow; Reissue edition,39.0,4.8,4
4,Devil's Claw: A Joanna Brady Mystery (Joanna B...,J. A. Jance,In the night-still canyons of the Arizona dese...,William Morrow; Reissue edition,63.0,4.5,5


In [12]:
book_list["text"] = book_list["title"].map(str) + " " + book_list["author"] +" "+ book_list["description"]+ " "+book_list['publisher']
book_list.head(2)

,title,author,description,publisher,customer_reviews,stars,id,text
0,A Time to Kill: A Novel [Kindle Edition],John Grisham,"Don’t miss a sneak peek of Sycamore Row, John ...",Dell,918.0,4.5,1,A Time to Kill: A Novel [Kindle Edition] John ...
1,Failure to Appear (J. P. Beaumont Novel) [Kind...,J. A. Jance,A shattering tale of tragedy and horror from t...,William Morrow; Reprint edition,34.0,4.8,2,Failure to Appear (J. P. Beaumont Novel) [Kind...


In [13]:
book_list_df = book_list[['id', 'text', 'title']]

In [14]:
book_list_df.head()

,id,text,title
0,1,A Time to Kill: A Novel [Kindle Edition] John ...,A Time to Kill: A Novel [Kindle Edition]
1,2,Failure to Appear (J. P. Beaumont Novel) [Kind...,Failure to Appear (J. P. Beaumont Novel) [Kind...
2,3,Twelve Years a Slave [Kindle Edition] Solomon ...,Twelve Years a Slave [Kindle Edition]
3,4,Without Due Process (J. P. Beaumont Novel) [Ki...,Without Due Process (J. P. Beaumont Novel) [Ki...
4,5,Devil's Claw: A Joanna Brady Mystery (Joanna B...,Devil's Claw: A Joanna Brady Mystery (Joanna B...


In [15]:
book_list_df.shape

(2063, 3)

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/audreymaldonado/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/audreymaldonado/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/audreymaldonado/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/audreymaldonado/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()


In [18]:

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   

Remove stop Words, Remove non-alphanumeric characters, lemmatize columns, merge all columns in order to create a corpus of text for each book

In [19]:
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.replace("kindle", "")
    text = text.replace("edition", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [20]:
book_list_df['text'] = book_list_df['text'].apply(clean_txt)

<ipython-input-20-f10be6f7747f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_list_df['text'] = book_list_df['text'].apply(clean_txt)


In [21]:
book_list_df.head()

,id,text,title
0,1,time kill novel kindle edition john grisham mi...,A Time to Kill: A Novel [Kindle Edition]
1,2,failure appear beaumont novel kindle edition j...,Failure to Appear (J. P. Beaumont Novel) [Kind...
2,3,twelve years slave kindle edition solomon nort...,Twelve Years a Slave [Kindle Edition]
3,4,without due process beaumont novel kindle edit...,Without Due Process (J. P. Beaumont Novel) [Ki...
4,5,devil claw joanna brady mystery joanna brady m...,Devil's Claw: A Joanna Brady Mystery (Joanna B...


Speculative word bank, hand-built, with inspiration from the Word2Vec model I built in the previous capstone. 

In [22]:
reader_choices = ['monster', 'lore', 'grave', 'animal', 'dread', 'haunted', 'claw', 'eerie', 'old', 'mystery', 'dark', 'fear', 'ghost', 'terror', 'blood', 'gore', 'death', 'danger', 'atmosphere', 'supernatural']

In [23]:
import random

In [52]:

sample_reader = random.sample(reader_choices, 3)

In [53]:
sample_reader = ' '.join([str(elem) for elem in sample_reader])

In [54]:
sample_reader

'old blood dark'

In [55]:
reader_number = 1

Different from a collaborative filtering approach which builds a model from a user's past behaviors, a Content based approach utilizes a series of discrete characteristics of an item in order to recommend additional items with similar properties. My implementation could be improved by adding data collectino features that would eventually turn it in to a hybrid approach. 

In [56]:
dict = {'user_id': [reader_number], 'selections': [sample_reader]}

In [57]:
user_df = pd.DataFrame(dict)

In [58]:
user_df

,user_id,selections
0,1,old blood dark


In [59]:
user_df['text'] = user_df['selections'].map(str).apply(clean_txt)

In [60]:
user_df

,user_id,selections,text
0,1,old blood dark,old blood dark


In [61]:
user_recc_df = user_df[['user_id', 'text']]

In [62]:
u = 1
index = np.where(user_recc_df['user_id']==u)[0][0]
user_q = user_recc_df.iloc[[index]]
user_q

,user_id,text
0,1,old blood dark


In [63]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_bookid = tfidf_vectorizer.fit_transform((book_list_df['text'])) #fitting and transforming the vector
tfidf_bookid

<2063x25213 sparse matrix of type '<class 'numpy.float64'>'
	with 242641 stored elements in Compressed Sparse Row format>

Using TF-IDF

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(user_q['text'])
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_bookid)

In [65]:
output2 = list(cos_similarity_tfidf)

In [66]:
def get_recommendation(top, book_list_df, scores):
  recommendation = pd.DataFrame(columns = ['title', 'score'])
  count = 0
  for i in top:
      recommendation.at[count, 'user_id'] = u
      recommendation.at[count, 'id'] = book_list_df['id'][i]
      recommendation.at[count, 'title'] = book_list_df['title'][i]
      recommendation.at[count, 'score'] =  scores[count]
      count += 1
  return recommendation

In [67]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]
list_scores = [output2[i][0][0] for i in top]
get_recommendation(top,book_list_df, list_scores)

,title,score,user_id,id
0,PULSE [Kindle Edition],0.199253,1.0,151.0
1,Dark Places: by Gillian Flynn -- Sidekick [Kin...,0.167221,1.0,1955.0
2,Anna Dressed in Blood (Anna Dressed in Blood S...,0.15792,1.0,2023.0
3,Life's Blood (Pulse Book 2) (PULSE Vampire Ser...,0.156698,1.0,358.0
4,Cold Blooded Killers (Killers from around the ...,0.156517,1.0,2008.0
5,A Perfect Blood (Hollows) [Kindle Edition],0.149126,1.0,574.0
6,The Memory of Blood: A Peculiar Crimes Unit My...,0.146766,1.0,1980.0
7,Blood Forest (Suspense thriller) [Kindle Edition],0.138879,1.0,1283.0
8,Silent Kills [Kindle Edition],0.136596,1.0,1682.0
9,The House Beyond the Hill [Kindle Edition],0.130562,1.0,458.0


Using spaCy

Will need to change where I put this spacy pkg


In [68]:
%%time
list_docs = []
for i in range(len(book_list_df)):
  doc = nlp("u'" + book_list_df['text'][i] + "'")
  list_docs.append((doc,i))
print(len(list_docs))

2063
CPU times: user 43 s, sys: 4.27 s, total: 47.2 s
Wall time: 47.6 s


In [69]:
def calculateSimWithSpaCy(nlp, df, user_text, n=6):
    # Calculate similarity using spaCy
    list_sim =[]
    doc1 = nlp("u'" + user_text + "'")
    for i in df.index:
      try:
            doc2 = list_docs[i][0]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, list_docs[i][1],score))
      except:
        continue

    return  list_sim

In [70]:
user_q.text[0]

'old blood dark'

In [71]:
%%time
 df3 = calculateSimWithSpaCy(nlp, book_list_df, user_q.text[0], n=15)

CPU times: user 1.21 s, sys: 11.3 ms, total: 1.22 s
Wall time: 1.22 s


In [72]:
df_recom_spacy = pd.DataFrame(df3).sort_values([3], ascending=False).head(10)

In [73]:

df_recom_spacy.reset_index(inplace=True)

In [74]:
index_spacy = df_recom_spacy[2]
list_scores = df_recom_spacy[3]

In [75]:
get_recommendation(index_spacy, book_list_df, list_scores)

,title,score,user_id,id
0,Flesh [Kindle Edition],0.720425,1.0,259.0
1,Her Master's Madness (A Dark Fairy Tale) [Kind...,0.715353,1.0,601.0
2,HORRORIFIC 3 Stories Even Colder and Darker [K...,0.71306,1.0,695.0
3,Sadistic Shorts: Tales from the Devil's Drawer...,0.71168,1.0,696.0
4,Anna Dressed in Blood (Anna Dressed in Blood S...,0.710371,1.0,2023.0
5,The Night the Moon Made a Sound [Kindle Edition],0.703213,1.0,740.0
6,Into the Darkness: An End of Days Zombie Thril...,0.701825,1.0,616.0
7,Dragged Into Darkness [Kindle Edition],0.698092,1.0,1729.0
8,Shiver (Wolves of Mercy Falls) [Kindle Edition],0.697562,1.0,816.0
9,PULSE [Kindle Edition],0.694092,1.0,151.0
